In [14]:
import torch
import numpy as np
from PIL import Image
from diffusers import LDMSuperResolutionPipeline
from hyperscope import config
from tqdm import tqdm

def minmax_norm(img):
    return (img - img.min())/(img.max() - img.min())

def split_image_into_patches(image, patch_size=128):
    """Split image into patches of size patch_size x patch_size."""
    if isinstance(image, Image.Image):
        width, height = image.size
        image = np.array(image).astype(np.uint8)
    else:
        height, width = image.shape[:2]
    
    patches = []
    positions = []
    
    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            # Handle edge cases where patch might be smaller than patch_size
            h = min(patch_size, height - y)
            w = min(patch_size, width - x)
            
            # If patch is smaller than patch_size, pad it
            patch = image[y:y+h, x:x+w]
            if h < patch_size or w < patch_size:
                if len(image.shape) == 3:  # RGB image
                    padded = np.zeros((patch_size, patch_size, image.shape[2]), dtype=np.uint8)
                else:  # Grayscale image
                    padded = np.zeros((patch_size, patch_size), dtype=np.uint8)
                padded[:h, :w] = patch
                patch = padded
            
            if isinstance(patch, np.ndarray):
                patch = Image.fromarray(patch)
            
            patches.append(patch)
            positions.append((x, y, w, h))  # Store original width and height for reconstruction
    
    return patches, positions

def reassemble_patches(patches, positions, original_size):
    """Reassemble patches back into a complete image."""
    width, height = original_size
    upscale_factor = 4  # LDM super-resolution upscales by 4x
    final_image = Image.new('RGB', (width * upscale_factor, height * upscale_factor))
    
    for patch, (x, y, orig_w, orig_h) in zip(patches, positions):
        # Calculate the size of the upscaled patch
        upscaled_w = orig_w * upscale_factor
        upscaled_h = orig_h * upscale_factor
        
        # If the patch was padded, we need to crop it back to the original aspect ratio
        if orig_w < 128 or orig_h < 128:
            patch = patch.crop((0, 0, upscaled_w, upscaled_h))
        
        # Paste the patch into the correct position
        final_image.paste(patch, (x * upscale_factor, y * upscale_factor))
    
    return final_image

def process_image(image_path, pipeline, patch_size=128):
    # Load and preprocess image
    img = np.load(image_path).astype(np.uint16)
    img = img[200:860, 117:1440]
    img = minmax_norm(img)
    img = Image.fromarray(img * 255).convert("RGB")
    
    # Get original size
    original_size = img.size
    
    # Split into patches
    patches, positions = split_image_into_patches(img, patch_size)
    
    # Process each patch
    upscaled_patches = []
    for patch in tqdm(list(patches)):
        upscaled_patch = pipeline(patch, num_inference_steps=2, eta=1).images[0]
        upscaled_patches.append(upscaled_patch)
    
    # Reassemble patches
    final_image = reassemble_patches(upscaled_patches, positions, original_size)
    
    return final_image

# Setup pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "CompVis/ldm-super-resolution-4x-openimages"
pipeline = LDMSuperResolutionPipeline.from_pretrained(model_id)
pipeline = pipeline.to(device)

# Process image
image_path = config.INTERIM_DATA_DIR / "worms" / "extracted" / "mkate_live_2025-01-13T10-31-47.631_3_page_25.npy"
final_image = process_image(image_path, pipeline)

# Save the final image
final_image.save("ldm_generated_image_full.png")


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/rich/.cache/huggingface/hub/models--CompVis--ldm-super-resolution-4x-openimages/snapshots/0b55ddf931a8e3a1b426b3a50ddcf325ff84f668/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/rich/.cache/huggingface/hub/models--CompVis--ldm-super-resolution-4x-openimages/snapshots/0b55ddf931a8e3a1b426b3a50ddcf325ff84f668/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /home/rich/.cache/huggingface/hub/models--CompVis--ldm-super-resolution-4x-openimages/snapshots/0b55ddf931a8e3a1b426b3a50ddcf325ff84f668/vqvae: Error no file named diffusion_pytorch_model.safetensors found in directory /home/rich/.cache/huggingface/hub/models--CompVis--ldm-super-resolution-4x-openimages/snapshots/0b55ddf931a8e3a1b426b3a50ddcf325ff84f668/vqvae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
  0%|          | 0

  0%|          | 0/2 [00:00<?, ?it/s]

  2%|▏         | 1/66 [00:14<16:03, 14.82s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

...

 97%|█████████▋| 64/66 [14:34<00:26, 13.45s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 98%|█████████▊| 65/66 [14:47<00:13, 13.46s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 66/66 [15:01<00:00, 13.66s/it]


In [ ]:
predictions